# Context Manager

In this example, we will create a database, use try..except..finally to manage the resources.  Then create a class to manage the context and remove the finally block.


In [9]:
import os
import sqlite3



## Remove the database if it is still there.
database = "contacts.db"

try:
    os.remove(database)
except FileNotFoundError:
    print("Thanks Sledge for a great conference.")

    
## Create the database using the typical try..catch..finally
con = sqlite3.connect(database)

create_sql = """
CREATE TABLE contacts (
    contact_id INTEGER PRIMARY KEY,
    first_name TEXT NOT NULL,
    last_name TEXT NOT NULL,
    email TEXT NOT NULL UNIQUE,
    phone TEXT NOT NULL UNIQUE
)

"""

try:
    cur = con.cursor()

    cur.execute(create_sql)
    cur.execute("INSERT INTO contacts VALUES (1062008,'Chris','Froome','chris@froome.com','6073418190')")

    con.commit()
    
    print("Contacts table created.  Ain't Sledge Grand?")

except sqlite3.Error as e:
    print(f"It hit the fan... {e.args[0]}")
finally:
    con.close()



Thanks Sledge for a great conference.
Contacts table created.  Ain't Sledge Grand?


In [56]:
## Using the context manager decorator to manage the connection (closing)
class ContactsDB():
    def __init__(self, file='contacts.db'):
        self.file=file

    def __enter__(self):
        self.conn = sqlite3.connect(self.file)
        self.conn.row_factory = sqlite3.Row
        return self.conn.cursor()
    
    def __exit__(self, exc_type, exc_value, exc_tb):
        if isinstance(exc_value, Exception):
            self.conn.rollback()
            print(f"Rolled back.")
        else:
            self.conn.commit()
            print(f"Committed.")
            
        self.conn.close()
        
        print("Connection closed.")

In [57]:
## Success
with ContactsDB() as cursor:
    results = cursor.execute('select first_name from contacts where last_name = ?', ("Froome",)).fetchall()
    print(results[0][0])

Chris
Committed.
Connection closed.


In [58]:
## Failure
with ContactsDB() as cursor:
    results = cursor.execute('select first_name from contacts where last_name = ?', ("Froome",)).fetchall()
    
    raise Exception("Oh no....")
    

Rolled back.
Connection closed.


Exception: Oh no....

In [47]:
## Using the context manager decorator

from contextlib import contextmanager

@contextmanager
def db_transaction ():
    connection = sqlite3.connect("contacts.db")
    cursor = connection.cursor()
    try:
        yield cursor
    except:
        connection.rollback()
        print("Rolled back.")
        
        raise
    else:
        connection.commit()
        print("Committed.")
        
    finally:
        connection.close()
        print("Connection Closed.")
        

In [48]:
## Success Usage
with db_transaction() as cursor:
    results = cursor.execute('select first_name from contacts where last_name = ?', ("Froome",)).fetchall()
    print(results[0][0])

Chris
Committed.
Connection Closed.


In [49]:
## Exception Usage
with db_transaction() as cursor:
    results = cursor.execute('select first_name from contacts where last_name = ?', ("Froome",)).fetchall()
    
    raise Exception("Oh no....")

Rolled back.
Connection Closed.


Exception: Oh no....

In [50]:
import csv

file_path = '/tmp/first_names.csv'

with db_transaction() as read_cursor, open(file_path, 'w', newline='') as csvfile:
    results = read_cursor.execute('select first_name from contacts where last_name = ?', ("Froome",)).fetchall()
    
    writer = csv.writer(csvfile)
    for result in results:
        fname = result[0]
        writer.writerow([fname])
        

print("\nReading the file...")
with open(file_path, "r") as reader:
    for line in reader:
        print(line)

Committed.
Connection Closed.

Reading the file...
Chris



## References

* RealPython (Great Article):  https://realpython.com/python-with-statement/
